# Anexos HPC

## Usando Cython fuera de Jupyter

El código cython se guarda tipicamente con extensión `pyx`

cuando hacemos `cython -3 codigo.pyx` se genera un código C usando la opción `-3` para indicar que se usará Python 3

In [ ]:
!cython -3 fractal_cython.pyx

Luego usamos nuestro compilador de C preferido apuntando adecuadamente a las cabeceras y librerías de Python 3 (y en este caso de NumPy también)

Debemos generar una libreria compartida (shared)

In [ ]:
!gcc -I/usr/include/python3.8m/ -I/usr/lib/python3.8/site-packages/numpy/core/include/ -L/usr/lib -lm -lpython3 -fPIC -shared fractal_cython.c -o fractal_cython.so

Luego podemos importar la librería como un módulo de Python cualquiera

In [ ]:
from fractal_cython import make_fractal_cython as make_fractal_imported
image_cython_imported = np.empty(shape=(N, 2*N), dtype=np.int64)
make_fractal_imported(N, image_cython_imported)
np.allclose(image_cython, image_cython_imported)

Limpiamos nuestros compilados si ya no son necesarios:

In [ ]:
!rm fractal_cython.c fractal_cython.so

## Compilación Just-in-time (JIT) con [Numba](http://numba.pydata.org/)

Podemos acelerar cálculos científicos de forma simple y semi-automática usando el compilador **Numba**. Este compilador no requiere cambiar el interprete de Python y tampoco es necesario aprender otro lenguaje

A través de decoradores podemos pedirle a Numba que compile una función "al vuelo" (just-in-time). Internamente Numba traduce las funciones de Python a lenguaje de máquina usando el compilador [LLVM](https://llvm.org/)

Para "ciertas funciones" el resultado será una versión compilada notoriamente más rápida en su ejecución que la original. Numba está diseñado para hacer más eficiente rutinas *compute-bound* que hagan **cálculos numéricos**. Tiene soporte para compilar funciones de Numpy y para paralelizar automaticamente ciclos `for`

Instala Numba en tu ambiente de conda con

    conda install numba

A continuación veremos los decoradores fundamentales de *Numba* y algunos ejemplos

Volvamos al código vectorizado para calcular la distancia euclidiana "todos con todos"

In [ ]:
import numpy as np

data = np.random.randn(1000, 2)

def distancia_pares_numpy(data):
    return np.sqrt(np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1))

%timeit -r3 -n1 distancia_pares_numpy(data)

Para usar Numba

- Importamos el decorador `jit` y lo aplicamos a la función anterior
- Usaremos el modo `nopython` o "modo rápido", esto indica al compilador que la función no usará el interprete de Python
- La primera llamada a la función es lenta, pues acciona el compilador
- La siguientes llamadas son más rápidas que la función de NumPy

In [ ]:
from numba import jit

@jit(nopython=True)
def distancia_pares_numba(data):
    return np.sqrt(np.sum((data.reshape(-1, 1, 2) - data.reshape(1, -1, 2))**2, axis=-1))

distancia_pares_numba(data) # Aquí se ejecuta la compilación

%timeit -r3 -n1 distancia_pares_numba(data)

np.allclose(distancia_pares_numpy(data), 
            distancia_pares_numba(data))

- Existe un alias para `jit(nopython=True)` llamado `njit`
- Otros argumentos interesantes para decorar son: `parallel=True` y `fastmath=True` 

**Una nota sobre NumPy**

No todas las [funcionalidades y funciones de NumPy están soportadas](https://numba.pydata.org/numba-doc/latest/reference/numpysupported.html)

Por ejemplo en la función `make_fractal_vectorized` al intentar numbificar existen varios errores

- `linspace` sólo soporta la versión con tres argumentos
- `repeat` no acepta el argumento `axis`
- *fancy-indexing* en arreglos 2D no está soportado por lo que `image[mask] +=1` no funciona

En este caso podriamos intentar "numbificar" la versión no vectorizada

In [ ]:
from numba import njit

@njit
def evaluate_numba(zi, zr, maxiters=50, cr=-0.835, ci=-0.2321):
    nit = 0
    zi2 = zi*zi
    zr2 = zr*zr
    while zi2 + zr2 <= 4. and nit < maxiters:
        zi = 2*zr*zi + ci
        zr = zr2 - zi2 + cr
        zr2 = zr*zr
        zi2 = zi*zi
        nit +=1
    return nit

@njit
def make_fractal_numba(N, maxiters=50):
    image = []
    for i in range(N):
        row = []
        for j in range(2*N):
            zi = -1.0 + i*2/N
            zr = -2.0 + j*2/N
            row.append(evaluate_numba(zi, zr, maxiters))
        image.append(row)
    return image


N = 2000
make_fractal_numba(N)
#%timeit -r3 -n1 make_fractal(N)
%timeit -r3 -n1 make_fractal_vectorized(N)
%timeit -r3 -n1 make_fractal_numba(N)
%timeit -r3 -n1 make_fractal_cython(N)